## CNN Grid Search

From our paper on "Explainable Prediction of Acute Myocardial Infarction using Machine Learning and Shapley Values"

In [9]:
import pandas as pd
import numpy as np
import pylab as plt
from keras import optimizers, losses, activations, models, regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, Flatten, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from keras.utils import to_categorical
from keras.models import load_model, Sequential
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix
from keras.wrappers.scikit_learn import KerasClassifier
from numpy import loadtxt
from sklearn.model_selection import GridSearchCV

In [10]:
# Load array
train = loadtxt('train.csv', delimiter=',')
test = loadtxt('test.csv', delimiter=',')

# Split array
train_x = train[:,:11]
test_x = test[:,:11]
train_y = train[:,11]
test_y = test[:,11]

In [23]:
def make_model(learning_rate=0.001):
    # number of categories in our problem
    opt = optimizers.Adam(learning_rate)
    model = Sequential()
    # shape of input
    model.add (Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid", input_shape=(11,1)))
    model.add (Dropout(rate=0.1))
    model.add (Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid"))
    model.add (Dropout(rate=0.1))
    model.add (Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid"))
    model.add (Dropout(rate=0.1))
    model.add (Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid"))
    model.add (GlobalMaxPool1D())
    model.add (Dropout(rate=0.1))
    model.add (Dense(64, activation=activations.relu, name="dense_1", kernel_regularizer=regularizers.l2(l=0.1)))
    model.add (Dense(16, activation=activations.relu, name="dense_2", kernel_regularizer=regularizers.l2(l=0.1)))
    model.add (Dense(2, activation=activations.softmax, name="dense_3_ecg_view"))
    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    return model

In [25]:
X_train = np.reshape(train_x, (train_x.shape[0], train_x.shape[1],1 ))
X_test = np.reshape(test_x, (test_x.shape[0], test_x.shape[1],1 ))

CNN = KerasClassifier(build_fn=make_model)

# Grid Search to optimize the model 
parameters = {'learning_rate':[0.001, 0.01, 0.1, 1]}
scoring = ['roc_auc', 'accuracy', 'f1']
clf = GridSearchCV(CNN, parameters, scoring, refit = 'roc_auc')
grid_result = clf.fit(X_train, train_y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 1/1
338424/338424 [==============================] - 38s 113us/step - loss: 0.4830 - acc: 0.8028
Epoch 1/1
338425/338425 [==============================] - 48s 140us/step - loss: 0.4817 - acc: 0.8042
Epoch 1/1
338425/338425 [==============================] - 44s 129us/step - loss: 0.4824 - acc: 0.8021
Epoch 1/1
338425/338425 [==============================] - 42s 125us/step - loss: 0.4891 - acc: 0.7971
Epoch 1/1
338425/338425 [==============================] - 40s 117us/step - loss: 0.4861 - acc: 0.8009
Epoch 1/1
338424/338424 [==============================] - 43s 127us/step - loss: 0.4582 - acc: 0.7906
Epoch 1/1
338425/338425 [==============================] - 41s 120us/step - loss: 0.4545 - acc: 0.7946
Epoch 1/1
338425/338425 [==============================] - 44s 129us/step - loss: 0.4577 - acc: 0.7918
Epoch 1/1
338425/338425 [==============================] - 43s 126us/step - loss: 0.4576 - acc: 0.7924
Epoch 1/1
338425/338425 [==============================] - 43s 127us/step